In [79]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import os

# Create an upload widget
uploader = widgets.FileUpload(
    accept='.csv,.json',
    multiple=False
)
display(uploader)


FileUpload(value=(), accept='.csv,.json', description='Upload')

In [80]:
def process_uploaded_file(uploader):
    if len(uploader.value) == 0:
        print("No file uploaded yet.")
        return None
    
    # Handle tuple format from ipywidgets
    upload_content = uploader.value[0] if isinstance(uploader.value, tuple) else next(iter(uploader.value.values()))
    
    # Extract filename and content based on the actual structure
    if 'metadata' in upload_content:
        filename = upload_content['metadata']['name']
        content = upload_content['content']
    else:
        # Alternative structure
        filename = upload_content.name
        content = upload_content.content

    # Save to temporary directory
    temp_filepath = f'temp/{filename}'
    os.makedirs('temp', exist_ok=True)
    with open(temp_filepath, 'wb') as f:
        f.write(content)
    
    # Load with pandas
    if filename.endswith('.csv'):
        df = pd.read_csv(temp_filepath)
    elif filename.endswith('.json'):
        df = pd.read_json(temp_filepath)
    else:
        raise ValueError('Unsupported file format')
    return df

# Usage:
# After uploading, run:
# df = process_uploaded_file(uploader)


### File Upload and Processing

1. Use the widget above to upload a `.csv` or `.json` file.
2. Then run:

```python
df = process_uploaded_file(uploader)
```

- If no file is uploaded yet, you'll see a message and `None` is returned.
- Once you have `df` (a pandas DataFrame), continue with your RAG pipeline: chunking, embedding, and vector DB steps.


In [81]:
# Index the uploaded DataFrame into Qdrant
from main import build_index_from_df

# First, process the uploaded file to create df
if 'uploader' in globals() and len(uploader.value) > 0:
    df = process_uploaded_file(uploader)
    if df is not None:
        build_index_from_df(df)
        print("Indexed uploaded DataFrame into Qdrant collection.")
    else:
        print("Failed to process uploaded file.")
else:
    print("No file uploaded yet. Use the upload widget above first.")


Indexed uploaded DataFrame into Qdrant collection.
